In [ ]:

    
@torch.no_grad()
def eval_obj_det(model, loader):
    global STEP
    global SAVE_METRIC
    model.eval()
    AP = [] #image_idx --> AP
    AP_50 = []
    AR_100 = []
    
    for batch_idx, sample in enumerate(loader):
        AP_dict = {}
        AP_50_dict= {}
        AR_100_dict= {}
        for class_name, boxes_list in sample['class_to_bbox'].items():
            instruction = f"\n\nDetect: {class_name}\n\n"
            out = model.detect(sample['image'], instruction)['objects']
            ######## TO DO --> plot 5-10 outputs
            metadata = {
                "size" : sample['size'],
                "prompt": instruction,
                'filename': sample['filename'],
            }
            metrics = per_object_mAP([_.tolist() for _ in boxes_list], [list(_.values()) for _ in out], metadata)
            AP_50_dict.update({class_name:float(metrics['AP_50'])})
            AR_100_dict.update({class_name:float(metrics['AR_100'])})
            AP_dict.update({class_name:float(metrics['AP'])})
        
        AP.append(AP_dict)
        AP_50.append(AP_50_dict)
        AR_100.append(AR_100_dict)

    _ = get_metric_summary(AP, 'AP (0.50:0.95)')
    avg_AP_50 = get_metric_summary(AP_50, 'AP (0.50)')
    avg_AR_100 = get_metric_summary(AR_100, 'AR (0.50:0.95) | 100 Det')
    
    if avg_AP_50 > SAVE_METRIC:
        SAVE_METRIC = avg_AP_50
        save_file(
        model.state_dict(),
        f"checkpoints/{CKPT_NAME}.safetensors",
        )

if torch.cuda.is_available():
    torch.set_default_device("cuda")

